### Monitoring Model

Real time model should already be deployed on Sagemaker.

##### Getting test data

In [1]:
import os

# Going to parent directory
os.chdir('..')
os.chdir('..')
os.getcwd()

'c:\\Users\\timot\\Documents\\DataScience\\WeCloudData\\Olyver_AI\\olyver_ai'

In [56]:
import pandas as pd
import numpy as np
from PIL import Image
import json 
from arize.pandas.embeddings import EmbeddingGenerator, UseCases
# Generator for creating image vectors
generator = EmbeddingGenerator.from_use_case(
    use_case=UseCases.CV.OBJECT_DETECTION,
    model_name="facebook/detr-resnet-101",
    batch_size=100
)
def coco_to_pascal_voc(coco_format: list):
    """ Annotations are in Coco format. Need to convert for Arize."""
    x1, y1, w, h = coco_format
    return [x1,y1, x1 + w, y1 + h]

directory = 'Barbell-small-2\\test\\'
file = open(directory + '_annotations.coco.json', 'r')
s = json.load(file)
test_df = pd.DataFrame()
test_df['filepath'] = [directory + info['file_name'] for info in s['images']]
test_df['image_vector'] = generator.generate_embeddings(
                            local_image_path_col=test_df['filepath'])
test_df['img_height'] = [info['height'] for info in s['images']]
test_df['img_width'] = [info['width'] for info in s['images']]
test_df['actual_category'] = [annot['category_id'] for annot in s['annotations']]
test_df['actual_bbox'] = [[list(coco_to_pascal_voc(annot['bbox']))] for annot in s['annotations']]
test_df['actual_category'] = test_df.actual_category.apply(lambda x: ['Barbell'] if x == 1 else 'barbell')
file.close()
test_df.head()

  arize.utils.logging | INFO | Downloading pre-trained model 'facebook/detr-resnet-101'
  arize.utils.logging | INFO | Downloading image processor
  arize.utils.logging | INFO | Generating embedding vectors


Map: 100%|██████████| 82/82 [00:37<00:00,  2.17 examples/s]


,filepath,image_vector,img_height,img_width,actual_category,actual_bbox
0,Barbell-small-2\test\go4_mp4-257_jpg.rf.004818...,"[-0.3310777246952057, 1.1678690910339355, 1.40...",640,640,[Barbell],"[[165, 337, 521, 529]]"
1,Barbell-small-2\test\go8_mp4-38_jpg.rf.0149136...,"[0.07991311699151993, 1.2661880254745483, 1.23...",640,640,[Barbell],"[[128, 366, 333, 495]]"
2,Barbell-small-2\test\go4_mp4-168_jpg.rf.01b770...,"[-0.278786838054657, 1.046907663345337, 1.5918...",640,640,[Barbell],"[[178, 393, 554, 610]]"
3,Barbell-small-2\test\go2_mp4-32_jpg.rf.02b2172...,"[-0.3601897656917572, 1.2231099605560303, 1.63...",640,640,[Barbell],"[[178, 382, 549, 589]]"
4,Barbell-small-2\test\go2_mp4-51_jpg.rf.05caa01...,"[-0.3640094995498657, 1.303786277770996, 1.448...",640,640,[Barbell],"[[196, 287, 565, 494]]"


In [57]:
# Sending each test image to sagemaker for inference
import boto3

test_sample = test_df.sample(frac=.2, ignore_index=True)
# Initialize new columns
test_sample['pred_category'] = [None] * len(test_sample)
test_sample['pred_bbox'] = [None] * len(test_sample)
test_sample['pred_score'] = [None] * len(test_sample)
for idx, file_path in enumerate(test_sample.filepath):
    with open(file_path, 'rb') as jpeg:
        jpeg_bytes = jpeg.read()
        sagemaker_client = boto3.client('sagemaker-runtime')
        response = sagemaker_client.invoke_endpoint(
            EndpointName='d2-serve',
            Body=jpeg_bytes,
            ContentType='image/jpeg',
            Accept="json"
        )
        response_dict = response['Body'].read()
        pred_dict = json.loads(response_dict)
        # Actual bboxes contain only one detected object
        # Keep only highest scoring object
    idxmax = pred_dict['scores'].index(max(pred_dict['scores']))
    test_sample.at[idx, 'pred_category'] = pred_dict['pred_classes'][idxmax]
    test_sample.at[idx, 'pred_bbox'] = [pred_dict['pred_boxes'][idxmax]]
    test_sample.at[idx, 'pred_score'] = [pred_dict['scores'][idxmax]]
test_sample['pred_category'] = test_sample.pred_category.apply(lambda x: ['Barbell'] if x == 1 else 'barbell')

## Prepare data to be sent to Arize

### Update the timestamps.

In [58]:
from datetime import datetime 
# Set timestamp to now
test_sample['prediction_ts'] = datetime.timestamp(datetime.now())

### Add prediction ids

In [59]:
import uuid 

def add_prediction_id(df):
    return [str(uuid.uuid4()) for _ in range(df.shape[0])]

test_sample['prediction_id'] = add_prediction_id(test_sample)
test_sample.head()

,filepath,image_vector,img_height,img_width,actual_category,actual_bbox,pred_category,pred_bbox,pred_score,prediction_ts,prediction_id
0,Barbell-small-2\test\go63_mp4-11_jpg.rf.9a3542...,"[0.2552233934402466, 1.3033944368362427, 0.813...",640,640,[Barbell],"[[252, 209, 313, 320]]",[Barbell],"[[252.3106689453125, 206.792236328125, 312.743...",[0.9954445362091064],1.705722e+09,c5d4d7a7-6394-4fcf-8936-6465543acdc8
1,Barbell-small-2\test\go1_mp4-144_jpg.rf.f05c34...,"[-0.052728064358234406, 1.2870899438858032, 1....",640,640,[Barbell],"[[191, 293, 368, 428]]",[Barbell],"[[188.4693603515625, 297.152099609375, 371.739...",[0.9974507689476013],1.705722e+09,989a2efa-224e-413f-978b-a3e6184a1826
2,Barbell-small-2\test\go4_mp4-249_jpg.rf.a99c88...,"[-0.30702802538871765, 1.0928428173065186, 1.5...",640,640,[Barbell],"[[178, 256, 539, 461]]",[Barbell],"[[164.2253875732422, 248.3498992919922, 536.41...",[0.9969261288642883],1.705722e+09,4d8e4774-53ab-4679-9769-a38e055a967e
3,Barbell-small-2\test\go1_mp4-167_jpg.rf.24b197...,"[-0.06599133461713791, 1.3537335395812988, 1.2...",640,640,[Barbell],"[[188, 367, 382, 503]]",[Barbell],"[[193.48289489746094, 364.2384948730469, 375.2...",[0.9930497407913208],1.705722e+09,045bedf2-46c7-4559-ba62-c83e43c3b049
4,Barbell-small-2\test\go75_mp4-14_jpg.rf.c0a994...,"[0.3812791407108307, 1.4361335039138794, 0.934...",640,640,[Barbell],"[[280, 253, 342, 366]]",[Barbell],"[[277.9753723144531, 253.63755798339844, 338.3...",[0.9932827949523926],1.705722e+09,362bf93f-8965-43cf-83cf-603343a7654d


### Send data to Arize

In [60]:
from arize.pandas.logger import Client
from arize.utils.types import (
    Schema, 
    Environments, 
    ModelTypes,
    EmbeddingColumnNames,
    ObjectDetectionColumnNames
)

SPACE_KEY = "b7bee33"
API_KEY = "d710f787158280c98cf"
arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)
model_id = "detectron2-barbell-detection"
model_version = "1.0"
model_type = ModelTypes.OBJECT_DETECTION
if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print("✅ Import and Setup Arize Client Done! Now we can start using Arize!")

✅ Import and Setup Arize Client Done! Now we can start using Arize!


In [61]:
# tags=[
#   "drift_type"
# ]
embedding_feature_column_names={
    "image_embedding": EmbeddingColumnNames(
        vector_column_name="image_vector"
    )
}
object_detection_prediction_column_names=ObjectDetectionColumnNames(
    bounding_boxes_coordinates_column_name="pred_bbox",
    categories_column_name="pred_category",
    scores_column_name="pred_score"
)
object_detection_actual_column_names=ObjectDetectionColumnNames(
    bounding_boxes_coordinates_column_name="actual_bbox",
    categories_column_name="actual_category",
)

# Define a Schema() object for Arize to pick up data from the correct columns for logging
schema = Schema(
    prediction_id_column_name="prediction_id",
    timestamp_column_name="prediction_ts",
    #tag_column_names=tags,
    embedding_feature_column_names=embedding_feature_column_names,
    object_detection_prediction_column_names=object_detection_prediction_column_names,
    object_detection_actual_column_names=object_detection_actual_column_names,
)

In [62]:
# Logging Production DataFrame
response = arize_client.log(
    dataframe=test_sample,
    schema=schema,
    model_id=model_id,
    model_version=model_version,
    model_type=model_type,
    environment=Environments.PRODUCTION,
)

# If successful, the server will return a status_code of 200
if response.status_code != 200:
    print(f"❌ logging failed with response code {response.status_code}, {response.text}")
else:
    print(f"✅ You have successfully logged training set to Arize")

  arize.utils.logging | INFO | Success! Check out your data at https://app.arize.com/organizations/QWNjb3VudE9yZ2FuaXphdGlvbjo1NzA1/spaces/U3BhY2U6NjAxNg==/models/modelName/detectron2-barbell-detection?selectedTab=dataIngestion
✅ You have successfully logged training set to Arize
